<a href="https://colab.research.google.com/github/james-cranley/gcolab/blob/main/tavi_ppm_logistic_regression_finalfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Useful Resources

Chapter 9 R4HDS (LR modelling) https://argoshare.is.ed.ac.uk/healthyr_book/chap09-h1.html

Code for evaluating LR models https://raw.githubusercontent.com/easonfg/cali_tutorial

LinkedIn Learning Video on modelling https://www.linkedin.com/learning/healthcare-analytics-regression-in-r/overview-of-modeling-process?u=2963594


# Prepare environment and load packages

In [25]:
# Prepare R in Python
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [26]:
# Set WD

%%R

wd <- '/content/gdrive/MyDrive/Colab Notebooks/tavi_ppm'
setwd(wd)

R[write to console]: Error in setwd(wd) : cannot change working directory
Calls: <Anonymous> -> <Anonymous> -> withVisible -> setwd




Error in setwd(wd) : cannot change working directory
Calls: <Anonymous> -> <Anonymous> -> withVisible -> setwd


In [27]:
# Install packages (this takes a few minutes, wait for the 'green tick' above)
%%R
install.packages('finalfit')
install.packages('googlesheets4')
install.packages('kableExtra')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/finalfit_1.0.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2678470 bytes (2.6 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [38]:
# Load packages
%%R

library(tidyverse)
library(finalfit)
library(googlesheets4)
library(kableExtra)

# Load in data from google sheet

In [40]:
# Load in data from google sheet
%%R

# Load in data using from the .csv version of the TAVI data
gs4_deauth()
df <- read_sheet("https://docs.google.com/spreadsheets/d/15J2PadknsXwGa33B237BYornN6bZ9u1dWJdIrZUtMOA/edit#gid=916763304")
glimpse(df)

R[write to console]: Reading from "tavi_for_import"

R[write to console]: Range "tavi_for_import"



Rows: 379
Columns: 26
$ age              <dbl> 82, 89, 82, 63, 90, 81, 67, 93, 83, 85, 76, 81, 90, …
$ sex              <chr> "F", "M", "F", "M", "M", "M", "M", "M", "M", "F", "M…
$ pre_avb          <dbl> NA, NA, 0, NA, NA, 0, 0, 1, 1, 0, 0, 1, NA, NA, 1, 0…
$ pre_avb_type     <dbl> NA, NA, 0, NA, NA, 0, 0, 1, 1, 0, 0, 1, NA, NA, 1, 0…
$ pre_rhythm       <chr> "AF", "AF", NA, "AF", "AF", NA, NA, NA, NA, NA, NA, …
$ pre_pr           <dbl> NA, NA, 168, NA, NA, 198, 182, 288, 240, 120, 178, 2…
$ pre_qrsd         <dbl> 128, 88, 88, 100, 82, 98, 116, 158, 92, 80, 86, 92, …
$ pre_qrsmorph     <chr> "LBBB", "normal", "normal", "normal", "normal", "nor…
$ intra_valve_type <chr> "Edwards New", "Edwards New", "Edwards New", "Edward…
$ intra_valve_size <dbl> 26, 23, 23, 26, 26, 29, 26, 29, 34, 29, 26, 26, 23, …
$ intra_viv        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ intra_avb        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0…
$ intra_lbbb       <dbl> 0, 0,

In [41]:
#Clean the data
%%R

# Correct typos
df$intra_valve_type <- gsub("CoreValve Evolut Pro", "CoreValve Evolut PRO", df$intra_valve_type)
df$intra_valve_type <- gsub("CoreValve Evolute Pro", "CoreValve Evolut PRO", df$intra_valve_type)
df$post_qrsmorph <- gsub("Normal", "", df$post_qrsmorph)
df$post_qrsmorph <- gsub("normal", "", df$post_qrsmorph)

# Make variables into factors
fact_cols <- c("sex", "pre_avb", "pre_avb_type", "pre_rhythm", "pre_qrsmorph", "intra_valve_type", "intra_valve_size", "intra_viv", "intra_avb", "intra_lbbb", "intra_avb1", "intra_avb2", "intra_avb3", "intra_tpw", "post_avb", "post_avb_type", "post_rhythm", "post_qrsmorph", "ppm")
df[fact_cols] <- lapply(df[fact_cols], as.factor)

# Make variables into numerics
num_cols <- c("post_pr", "post_qrsd", "post_interval")
df[num_cols] <- lapply(df[num_cols], as.numeric)

# Make Edwards Old the reference for valve type
df <- within(df, intra_valve_type <- relevel(intra_valve_type, ref = "Edwards Old"))

glimpse(df)

R[write to console]: Error in lapply(df[num_cols], as.numeric) : 
  'list' object cannot be coerced to type 'double'
Calls: <Anonymous> -> <Anonymous> -> withVisible -> lapply




Error in lapply(df[num_cols], as.numeric) : 
  'list' object cannot be coerced to type 'double'
Calls: <Anonymous> -> <Anonymous> -> withVisible -> lapply


In [44]:
# Use FinalFit package to start making models

# Start with a model using only pre_tavi factors
%%R

dependent <- "ppm"
explanatory_pre <- c("age", "sex", "pre_avb", "pre_pr", "pre_qrsd")
pre <- df %>% finalfit(dependent, explanatory_pre, metrics = TRUE)
pre

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



[[1]]
 Dependent: ppm                      0            1           OR (univariable)
            age Mean (SD)   81.4 (7.5)   83.0 (6.9)  1.03 (0.98-1.09, p=0.226)
            sex         F   150 (89.8)    17 (10.2)                          -
                        M   195 (92.0)     17 (8.0)  0.77 (0.38-1.57, p=0.466)
        pre_avb         0   183 (94.8)     10 (5.2)                          -
                        1    67 (79.8)    17 (20.2) 4.64 (2.06-11.00, p<0.001)
         pre_pr Mean (SD) 185.3 (34.7) 220.8 (46.3)  1.02 (1.01-1.03, p<0.001)
       pre_qrsd Mean (SD) 107.6 (21.5) 120.1 (25.9)  1.02 (1.01-1.04, p=0.002)
        OR (multivariable)
 1.00 (0.95-1.07, p=0.888)
                         -
 0.51 (0.19-1.33, p=0.172)
                         -
 1.33 (0.34-4.99, p=0.677)
 1.02 (1.00-1.04, p=0.017)
 1.02 (1.00-1.04, p=0.066)

[[2]]
                                                                                                                                  
 Number 

In [45]:
# Now a model using intra-tavi factors
%%R

dependent <- "ppm"
explanatory_intra <- c("intra_valve_type", "intra_valve_size", "intra_avb", "intra_lbbb", "intra_avb1", "intra_avb2", "intra_avb3", "intra_tpw")
intra <- df %>% finalfit(dependent, explanatory_intra, metrics = TRUE)
intra

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



[[1]]
   Dependent: ppm                               0         1
 intra_valve_type CoreValve Evolut PRO  50 (83.3) 10 (16.7)
                           Edwards New  63 (95.5)   3 (4.5)
                           Edwards Old 232 (91.7)  21 (8.3)
 intra_valve_size                   20  11 (91.7)   1 (8.3)
                                    23 106 (89.1) 13 (10.9)
                                    26 152 (93.3)  11 (6.7)
                                    29  71 (89.9)  8 (10.1)
                                    34   5 (83.3)  1 (16.7)
        intra_avb                    0 311 (96.6)  11 (3.4)
                                     1  34 (59.6) 23 (40.4)
       intra_lbbb                    0 323 (92.8)  25 (7.2)
                                     1  22 (71.0)  9 (29.0)
       intra_avb1                    0 335 (92.0)  29 (8.0)
                                     1  10 (66.7)  5 (33.3)
       intra_avb2                    0 343 (91.2)  33 (8.8)
                                  

In [46]:
# Now a model using post-tavi factors
%%R

dependent <- "ppm"
explanatory_post <- c("post_avb")
post <- df %>% finalfit(dependent, explanatory_post, metrics = TRUE)
post

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



[[1]]
 Dependent: ppm            0         1            OR (univariable)
       post_avb 0 154 (98.1)   3 (1.9)                           -
                1  97 (82.2) 21 (17.8) 11.11 (3.71-47.94, p<0.001)
          OR (multivariable)
                           -
 11.11 (3.71-47.94, p<0.001)

[[2]]
                                                                                                                                  
 Number in dataframe = 379, Number in model = 275, Missing = 104, AIC = 144.2, C-statistic = 0.744, H&L = Chi-sq(8) 0.00 (p=1.000)



In [47]:
# Now a model using all the factors
%%R

dependent <- "ppm"
explanatory_all <- c(explanatory_pre, explanatory_intra, explanatory_post)
df %>% finalfit(dependent, explanatory_all, metrics = TRUE)

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Waiting for profiling to be done...

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases



[[1]]
   Dependent: ppm                                 0            1
              age            Mean (SD)   81.4 (7.5)   83.0 (6.9)
              sex                    F   150 (89.8)    17 (10.2)
                                     M   195 (92.0)     17 (8.0)
          pre_avb                    0   183 (94.8)     10 (5.2)
                                     1    67 (79.8)    17 (20.2)
           pre_pr            Mean (SD) 185.3 (34.7) 220.8 (46.3)
         pre_qrsd            Mean (SD) 107.6 (21.5) 120.1 (25.9)
 intra_valve_type CoreValve Evolut PRO    50 (83.3)    10 (16.7)
                           Edwards New    63 (95.5)      3 (4.5)
                           Edwards Old   232 (91.7)     21 (8.3)
 intra_valve_size                   20    11 (91.7)      1 (8.3)
                                    23   106 (89.1)    13 (10.9)
                                    26   152 (93.3)     11 (6.7)
                                    29    71 (89.9)     8 (10.1)
                   

In [35]:
# Now reduce variables iteratively to improve AIC and c statistic
%%R

dependent <- "ppm"
explanatory_all <- c("post_avb", "intra_valve_type", "intra_valve_size", "intra_avb", "intra_lbbb", "intra_avb1", "intra_avb2", "intra_avb3", "intra_tpw", "age", "sex", "pre_avb", "pre_pr", "pre_qrsd")
explanatory_multi <- c("post_avb", "intra_valve_type", "intra_avb", "intra_lbbb", "intra_avb1", "intra_avb2", "intra_avb3", "intra_tpw", "pre_avb", "pre_pr", "pre_qrsd")
table1 <- df %>% finalfit(dependent, explanatory_all, explanatory_multi, metrics = TRUE)
kbl(table1)

R[write to console]: Error in finalfit(., dependent, explanatory_all, explanatory_multi, metrics = TRUE) : 
  .data is not dataframe
Calls: <Anonymous> -> <Anonymous> -> withVisible -> %>% -> finalfit




Error in finalfit(., dependent, explanatory_all, explanatory_multi, metrics = TRUE) : 
  .data is not dataframe
Calls: <Anonymous> -> <Anonymous> -> withVisible -> %>% -> finalfit
